In [50]:
import pandas as pd
import numpy as np 
import math

## Question E (Q2 and Q3 of TD5) 

In [23]:
df=pd.read_excel("Dataset TD5.xlsx",header=2)

In [5]:
df.head()

,Date,HIGH,LOW,Unnamed: 3,Date.1,HIGH.1,LOW.1,Unnamed: 7,Date.2,HIGH.2,LOW.2
0,2016-03-07 08:59:59.990,1.2932,1.2917,NaN,2016-03-07 08:59:59.990,0.10725,0.10720,NaN,2016-03-07 08:59:59.990,0.6842,0.6829
1,2016-03-07 09:15:00.000,1.2940,1.2930,NaN,2016-03-07 09:15:00.000,0.10728,0.10717,NaN,2016-03-07 09:15:00.000,0.6849,0.6841
2,2016-03-07 09:30:00.000,1.2943,1.2922,NaN,2016-03-07 09:30:00.000,0.10726,0.10719,NaN,2016-03-07 09:30:00.000,0.6844,0.6837
3,2016-03-07 09:45:00.000,1.2930,1.2913,NaN,2016-03-07 09:45:00.000,0.10728,0.10721,NaN,2016-03-07 09:45:00.000,0.6844,0.6839
4,2016-03-07 10:00:00.000,1.2931,1.2921,NaN,2016-03-07 10:00:00.000,0.10725,0.10719,NaN,2016-03-07 10:00:00.000,0.6840,0.6835


In [25]:
# Groupe 1 : GBPEUR (Colonnes 0, 1, 2)
df_gbp = df.iloc[:, 0:3].copy()
df_gbp.columns = ['Date', 'HIGH', 'LOW'] # Renommage pour être sûr
df_gbp['Currency'] = 'GBPEUR' # Optionnel : ajouter une étiquette

# Groupe 2 : SEKEUR (Colonnes 4, 5, 6) -> On saute la colonne 3 qui est vide
df_sek = df.iloc[:, 4:7].copy()
df_sek.columns = ['Date', 'HIGH', 'LOW']
df_sek['Currency'] = 'SEKEUR'

# Groupe 3 : CADEUR (Colonnes 8, 9, 10) -> On saute la colonne 7 qui est vide
df_cad = df.iloc[:, 8:11].copy()
df_cad.columns = ['Date', 'HIGH', 'LOW']
df_cad['Currency'] = 'CADEUR'

# 3. Vérification des résultats
print("Aperçu GBPEUR :")
print(df_gbp.head())
print("\nAperçu SEKEUR :")
print(df_sek.head())

Aperçu GBPEUR :
                     Date    HIGH     LOW Currency
0 2016-03-07 08:59:59.990  1.2932  1.2917   GBPEUR
1 2016-03-07 09:15:00.000  1.2940  1.2930   GBPEUR
2 2016-03-07 09:30:00.000  1.2943  1.2922   GBPEUR
3 2016-03-07 09:45:00.000  1.2930  1.2913   GBPEUR
4 2016-03-07 10:00:00.000  1.2931  1.2921   GBPEUR

Aperçu SEKEUR :
                     Date     HIGH      LOW Currency
0 2016-03-07 08:59:59.990  0.10725  0.10720   SEKEUR
1 2016-03-07 09:15:00.000  0.10728  0.10717   SEKEUR
2 2016-03-07 09:30:00.000  0.10726  0.10719   SEKEUR
3 2016-03-07 09:45:00.000  0.10728  0.10721   SEKEUR
4 2016-03-07 10:00:00.000  0.10725  0.10719   SEKEUR


In [9]:
# 1. Compute Average for GBPEUR
df_gbp['Average'] = (df_gbp['HIGH'] + df_gbp['LOW']) / 2

# 2. Compute Average for SEKEUR
df_sek['Average'] = (df_sek['HIGH'] + df_sek['LOW']) / 2

# 3. Compute Average for CADEUR
df_cad['Average'] = (df_cad['HIGH'] + df_cad['LOW']) / 2

In [11]:
# Haar Function
def haar(t):
    res = 0
    if 1 / 2 > t >= 0:
        res = 1
    elif 1 > t >= 1 / 2:
        res = -1
    return res

In [13]:
# Haar wavelet function
def waveletHaar(j, k, t):
    return 2 ** (j / 2) * haar(t * 2 ** j - k)

In [15]:
# Haar wavelet transform
def haarWaveletTransform(x, j, k):
    n = len(x)
    res = 0
    for u in range(1, n):
        res += waveletHaar(j, k, x[u]) * (x[u] - x[u - 1]) / x[u - 1]
    return res

In [16]:
# Covariance between two series
def cov(j, T, serie1, serie2):
    sum = 0

    s1 = 0
    s2 = 0

    for l in range(1, T):
        s1 += haarWaveletTransform(serie1, j, l)
        s2 += haarWaveletTransform(serie2, j, l)

    for k in range(1, T):
        sum += (haarWaveletTransform(serie1, j, k) - s1 / T) * (haarWaveletTransform(serie2, j, k) - s2 / T)

    return sum / T

In [ ]:
# Multiresolution correlation between GBP/EUR and SEK/EUR
j = 5
T = len(df_gbp)
correlation_gbp_sek = cov(j, T, df_gbp['Average'], df_sek['Average'])

# Multiresolution correlation between GBP/EUR and CAD/EUR
j = 5
T = len(df_gbp)
correlation_gbp_cad = cov(j, T, df_gbp['Average'], df_cad['Average'])

# Multiresolution correlation between SEK/EUR and CAD/EUR
j = 5
T = len(df_sek)
correlation_sek_cad = cov(j, T, df_sek['Average'], df_cad['Average'])

print("Multiresolution correlation between GBP/EUR and SEK/EUR: {:.20f}".format(correlation_gbp_sek))
print("Multiresolution correlation between GBP/EUR and CAD/EUR: {:.20f}".format(correlation_gbp_cad))
print("Multiresolution correlation between SEK/EUR and CAD/EUR: {:.20f}".format(correlation_sek_cad))

In [27]:
df_gbp['z(t)'] = (df_gbp['HIGH'] + df_gbp['LOW']) / 2
df_sek['z(t)'] = (df_sek['HIGH'] + df_sek['LOW']) / 2
df_cad['z(t)'] = (df_cad['HIGH'] + df_cad['LOW']) / 2

In [29]:
j = -7

for d in [df_gbp, df_sek, df_cad]:
    
    # t : Index temporel de 0 à N
    d['t'] = range(len(d))
    
    # k : t - 1
    d['k'] = d['t'] - 1
    
    # Colonnes dyadiques
    d['2^-jk'] = (2**(-j)) * d['k']
    d['2^-j(k+1)'] = (2**(-j)) * (d['k'] + 1)
    d['t(2^j)-1']=(2**(j))*d['t'] -1
    d['t(2^j)']=(2**(j))*d['t']

# 4. Affichage du résultat pour vérification
print(df_gbp.head())

                     Date    HIGH     LOW Currency     z(t)  t  k  2^-jk  \
0 2016-03-07 08:59:59.990  1.2932  1.2917   GBPEUR  1.29245  0 -1   -128   
1 2016-03-07 09:15:00.000  1.2940  1.2930   GBPEUR  1.29350  1  0      0   
2 2016-03-07 09:30:00.000  1.2943  1.2922   GBPEUR  1.29325  2  1    128   
3 2016-03-07 09:45:00.000  1.2930  1.2913   GBPEUR  1.29215  3  2    256   
4 2016-03-07 10:00:00.000  1.2931  1.2921   GBPEUR  1.29260  4  3    384   

   2^-j(k+1)  t(2^j)-1    t(2^j)  
0          0 -1.000000  0.000000  
1        128 -0.992188  0.007812  
2        256 -0.984375  0.015625  
3        384 -0.976562  0.023438  
4        512 -0.968750  0.031250  


In [33]:
for df in [df_gbp, df_sek, df_cad]:
    # Calcul du rendement logarithmique : ln( z(t) / z(t-1) )
    # z(t) est le prix actuel, z(t).shift(1) est le prix précédent
    df['returns'] = np.log(df['z(t)'] / df['z(t)'].shift(1))

In [35]:
resultats = []
avg_val = df_cad["returns"].values
for k in df_cad["k"]:
    borne_inf = k * 2**(-j)        
    borne_sup =  (k + 1) * 2**(-j)  
    
    # C'est l'équivalent de [indicatrice(A) - indicatrice(B)]
    masque = (df_cad["t"] >= borne_inf) & (df_cad["t"] < borne_sup)
    
    somme_locale = np.sum(avg_val[masque])
    resultats.append(2**(j/2) * somme_locale)

df_cad["coef wavelet returns"] = resultats

In [37]:
df_cad.head()

,Date,HIGH,LOW,Currency,z(t),t,k,2^-jk,2^-j(k+1),t(2^j)-1,t(2^j),returns,coef wavelet returns
0,2016-03-07 08:59:59.990,0.6842,0.6829,CADEUR,0.68355,0,-1,-128,0,-1.000000,0.000000,NaN,0.000000
1,2016-03-07 09:15:00.000,0.6849,0.6841,CADEUR,0.68450,1,0,0,128,-0.992188,0.007812,0.001389,NaN
2,2016-03-07 09:30:00.000,0.6844,0.6837,CADEUR,0.68405,2,1,128,256,-0.984375,0.015625,-0.000658,0.001538
3,2016-03-07 09:45:00.000,0.6844,0.6839,CADEUR,0.68415,3,2,256,384,-0.976562,0.023438,0.000146,-0.001400
4,2016-03-07 10:00:00.000,0.6840,0.6835,CADEUR,0.68375,4,3,384,512,-0.968750,0.031250,-0.000585,0.000463


Question 3

In [40]:
for df in [df_gbp, df_sek, df_cad]:
    # Calcul du rendement logarithmique : ln( z(t) / z(t-1) )
    # z(t) est le prix actuel, z(t).shift(1) est le prix précédent
    df['X'] = np.log(df['z(t)'])

df_gbp.head()

,Date,HIGH,LOW,Currency,z(t),t,k,2^-jk,2^-j(k+1),t(2^j)-1,t(2^j),coef wavelet(j=-7),returns,X
0,2016-03-07 08:59:59.990,1.2932,1.2917,GBPEUR,1.29245,0,-1,-128,0,-1.000000,0.000000,0.000000,NaN,0.256540
1,2016-03-07 09:15:00.000,1.2940,1.2930,GBPEUR,1.29350,1,0,0,128,-0.992188,0.007812,14.623459,0.000812,0.257352
2,2016-03-07 09:30:00.000,1.2943,1.2922,GBPEUR,1.29325,2,1,128,256,-0.984375,0.015625,14.625085,-0.000193,0.257158
3,2016-03-07 09:45:00.000,1.2930,1.2913,GBPEUR,1.29215,3,2,256,384,-0.976562,0.023438,14.545058,-0.000851,0.256307
4,2016-03-07 10:00:00.000,1.2931,1.2921,GBPEUR,1.29260,4,3,384,512,-0.968750,0.031250,14.581691,0.000348,0.256656


In [42]:
M2=0
n=range(len(d))
j=0
for i in range (252*2):
    M2=M2+np.abs(df_gbp["X"]*(i/252) - df_gbp["X"]*((i-1)/252))**2
M2=M2*(1/252*2)
M2=np.sum(M2)

In [44]:
M2

0.03819218890150894

In [60]:
M2_prime=0
n=range(len(d))
for i in range (252):
    M2_prime=M2_prime+np.abs(df_gbp["X"]*(2*i/252) - df_gbp["X"]*(2*(i-1)/252))**2
M2_prime=M2_prime*(1/252)
M2_prime=np.sum(M2_prime)

In [61]:
M2_prime

0.03819218890150895

In [64]:
H=(1/2)*math.log2(M2_prime/M2)

In [66]:
H

3.203426503814917e-16